##**Reto Quincenal - Árboles de decisión para predecir que jugadores son probables a realizar compras en la aplicaciòn**

Este modelo tiene como objetivo desarrolar un  de arbol de decision para facilitar la prediccion  de jugadores  de una aplicacion que tienen mayor probabilidad de realizar compras en la aplicacion, para asi poder tener una mejor segmentacion de los jugadores.

Las variables de entrada son:
* Age:Indica la edad  del jugador de la aplicacion
* Income: ingreso promedio del jugador de la aplicacion
* Years:  Indica cuantos años lleva el jugador utilizando la aplicacion
* Hours: Indica  el numero de horas que un jugador pasa usando la plataforma

La variable de salida o referencia es:
* Buy: Indica si el jugador a realizado alguna compra dentro de la aplicacion o no



**1. se cargan las librerías de trabajo**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier  #modelo de arbol de decisión
from sklearn.metrics import confusion_matrix
#carga automática de las bases de datos
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**2. Se procede con la carga de los datos**

In [ ]:
nxl = '/content/drive/MyDrive/Analítica de Negocios/2. BD2_In_App_Pur.xlsx'
XDB = pd.read_excel(nxl , sheet_name='In_App_Pur_Data') #Base de datos In_App_Pur_Data, lee la base de dato
XDB = XDB.dropna()  #Elimina celdas erroneas o N/A

XDB = XDB[['Age' , 'Income' , 'Years' , 'Hours' , 'Buy']]
#Separa variables de entrada y variables de salida
XD = XDB[['Age' , 'Income' , 'Years' , 'Hours']] # variables entrada
yd = XDB['Buy'] #variables de referencia - salida

**3. se procede con la implementación del modelo**

In [ ]:
mar = DecisionTreeClassifier(criterion = 'gini' , max_depth = 4) #max depth corresponde el maximo nivel de profundidad que tiene el arbol (5 porque tengo 5 variables)
mar.fit(XD , yd)

#para la grafica del arbol de decision
from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data
dot_data = export_graphviz(mar , feature_names = XD.columns)
graph = graph_from_dot_data(dot_data)
graph.write_png('2. Desition_Tree_Reto_Quincenal_2.png')

True

**4. evaluamos el comportamiento del modelo frente a los datos presentados**

In [ ]:
ydp = mar.predict(XD) #vamos a estresar el modelo - backtesting
cm = confusion_matrix(yd , ydp)
print(cm)
VN = cm[0,0]
FP = cm[0,1]
FN = cm[1,0]
VP = cm[1,1]
print('VN = ' , VN)
print('FP = ' , FP)
print('FN = ' , FN)
print('VP = ' , VP)
TD = VN + FP + FN + VP

#EXACTITUD: MUESTRA EL COMPORTAMIENTO GENERAL DEL MODELO
Ex = (VN+VP)/TD ; print("exactitud: " , Ex) #comportamiento general del modelo

#tasa de error: porcentaje de equivocacion
Ter= (FP+FN)/TD ; print("tasa de error: " , Ter) #comportamiento erroneo del modelo

#sensibilidad: como se comporta el modelo frente a los jugadores que compran en la pagina
Sen = VP/(VP+FN) ; print("sensibilidad: " , Sen) #comportamiento frente a los preaprobados Con VP y FP

#Especificidad: como se comporta el modelo frente a los que no compran en la pagina
Esp= VN/(VN+FP) ; print("especificidad: " , Esp) #comportamiento frente a los prenegados Con VN y FN

#precision: capacidad del modelo para predecir VP
prec = VP/(VP+FP) ; print('Precisión' , prec) #comportamiento solamente contando los positivos vp (lo que compran juegos en la pagina)

PredNeg = VN/(VN+FN)  ; print('Predictividad negativa' , PredNeg)#comportamiento solamente contando los negativos VN (lo que no compran juegos en la pagina)


[[139  50]
 [ 11 200]]
VN =  139
FP =  50
FN =  11
VP =  200
exactitud:  0.8475
tasa de error:  0.1525
sensibilidad:  0.9478672985781991
especificidad:  0.7354497354497355
Precisión 0.8
Predictividad negativa 0.9266666666666666


**5. Con base al arbol de decision creado determino las reglas  de los nodos puros**



In [ ]:
#Ruta Nodo Puro 1=Income<=63.5 en donde la decision se denota [90,0]
#Ruta Nodo Puro 2=Income>=63.5 AND years<=1.5 AND age>=23.0 en donde la decision se denota [28,0]
#Ruta Nodo Puro 3= Income>=63.5 AND years<=1.5 AND years<=0.5 AND Age >=23.0  en donde la decision se denota [5,0]

**6. Con base al arbol de decision creado determino cuál es la ruta de negocio y la probabilidad de que el primer jugador y el segundo jugador de la base de datos "In_App_Pur_Score" hagan compras dentro de la aplicación.**

In [ ]:
# Ruta de negocio 1er jugador = Income > 63.5 AND Years > 1.5AND Years <= 5.5 AND Income > 76.5 THEN [38,77]
#Gini= 0.442
#Sample= 115
ProbabilidadNoCompra_1er_jugador = (38/115)*100
probabilidadCompra_1er_jugador = (77/115)*100

# Ruta de negocio 2do jugador = Income > 63.5 AND Years <= 1.5 AND Age > 23.0 THEN [28,0]
#Gini= 0.0
#Sample= 28
probabilidadNoCompra_2do_jugador = (28/28)*100
probabilidadCompra_2do_jugador = (0/28)*100

print("La probabilidad de que el jugador 1 no haga una compra es de: " , ProbabilidadNoCompra_1er_jugador , "%")
print("La probabilidad de que el jugador 1 haga una compra es de: " , probabilidadCompra_1er_jugador , "%")
print("La probabilidad de que el jugador 2 no haga una compra es de: " , probabilidadNoCompra_2do_jugador, "%")
print("La probabilidad de que el jugador 2 haga una compra es de: " , probabilidadCompra_2do_jugador, "%" )


La probabilidad de que el jugador 1 no haga una compra es de:  33.04347826086956 %
La probabilidad de que el jugador 1 haga una compra es de:  66.95652173913044 %
La probabilidad de que el jugador 2 no haga una compra es de:  100.0 %
La probabilidad de que el jugador 2 haga una compra es de:  0.0 %


### **Analisis de resultados **

* De la matriz de confusion se puede observar que el modelo tiene la tendecnia a equivocarse en mayor medida en la prediccion de los valores de los jugadores que no compran en la pagina con un total de 11 falsos negativos, frente a un 50 de falsos positivos.

* La tasa de error del modelo corresponde a un 15%

 * De acuerdo con los resultados obtenidos por el modelo podemos observar de manera general que el modelo logro una exactitud que se ubico por encima del 84% en promedio (0.8475). Lo que indica el buen comportamiento general del modelo frente a la prediccion de si un jugador compra en la plataforma o no.

 * De esta misma manera se puede observar que el modelo tiende a predecir  mas jugadores que compram en la plataforma que los que no compram en la pagina, ya que tenemos un sensibilidad del 94% en promedio (0.9478) y una especifidad el 73% en promedio (0.7354)

 * Se destaca que en el arbol, que las variables que tuvieron una mayor relevancia fueron las variables Income, seguida de los Years.

 * Se destaca igualmente que el arbol tuvo un total de 3 nodos puros, el primero esta definido por la siguiente regla del negocio:Income<=63.5 en donde la decision se denota [90,0], lo que indica que los jugadores que cumplan con esta caracteristica no compran en la pagina con una certeza del 100%. El segundo nodo puro esta definido por la siguiente regla del negocio:Income>=63.5 AND years<=1.5 AND age>=23.0 en donde la decision se denota [28,0]lo que indica que los jugadores que cumplan con esta caracteristica no compran en la pagina con una certeza del 100%. Y el tercer nodo puro esta definido por la siguiente regla del negocio:Income>=63.5 AND years<=1.5 AND years<=0.5 AND Age >=23.0  en donde la decision se denota [5,0]lo que indica que los jugadores que cumplan con esta caracteristica no compran en la pagina con una certeza del 100%.

 * Por otro lado, podemos decir que una persona con las siguientes caracteristicas(Age=21, Income=128, Years=3, Hours=22) tiene la siguente ruta de negocio en el arbo de decision:Income > 63.5 AND Years > 1.5AND Years <= 5.5 AND Income > 76.5 en donde la decision se denota [38,77], Lo que nos permite conocer que la probabilidad de que este jugador compre en la plataforma es del 66.95% y de que no haga una compra en la plataforma del 33.04%

* Tambien, podemos decir que una persona con las siguientes caracteristicas(Age=50, Income=128, Years=1, Hours=6) tiene la siguente ruta de negocio en el arbo de decision:Income > 63.5 AND Years <= 1.5 AND Age > 23.0 en donde la decision se denota[28,0], Lo que nos permite conocer que la probabilidad de que este jugador compre en la plataforma es del 0.0% y de que no haga una compra en la plataforma del 100%

* Finalmente, podemos determinar el porcentaje que una persona  queutilice la tarjeta de crédito para la siguiente regla SI Income>63.5 AND Years>1.5 AND Years>5.5 AND Income<=137.5. en donde la decision se denota [11,21], esto nos permite saber que esta persona la probabilidad del 8% de que no compre en la pagina y una probabilidad del 91% de que si compre en la pagina